In [1]:
import asyncio
import nest_asyncio

nest_asyncio.apply()

In [2]:
queues_list = ["ticker", "trade", "minTicker", "depth", "aggTrade", "kline_ws", "execution_ws", "kline_fetcher", "orderbook_fetcher", "fetch_all_storage", "send_exponential", "fetch_exponential", "send_analysis", "fetch_analysis", "fetch_analysis", "send_analysis", "send_trading_status", "fetch_trading_status"]
events_list = ["stop_loop", "event_timer_start", "event_start_exponential", "event_done_exeponential", "event_request_receiver_data", "event_start_analysis", "event_done_analysis", "event_request_computed_results", "event_start_orders", "event_done_orders", "event_request_status", "event_start_monitor", "event_done_monitor", "event_request_message", "event_start_message", "event_done_message"]

In [ ]:
queues = {}
for queue in queues_list:
    queues[queue] = asyncio.Queue()
events = {}
for event in events_list:
    events[event] = asyncio.Event()

NameError: name 'queue_list' is not defined